## Question 6
`Write a query to retrieve the total revenue generated by each genre, but exclude any revenue from tracks with a duration of less than 30 seconds.`

(Hint: use a subquery with a NOT IN clause)

In [1]:
%run ../utils/setup_notebook.ipynb
from utils.format_sql_result import format_sql_result

In [2]:
%%sql 

SELECT *
FROM invoices
LIMIT 1;

InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98


In [16]:
%%sql 

SELECT *
FROM invoice_items
LIMIT 1;

InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
1,1,2,0.99,1


In [3]:
%%sql 

SELECT *
FROM tracks
LIMIT 1;

TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99


In [4]:
%%sql 

SELECT *
FROM genres
LIMIT 1;

GenreId,Name
1,Rock


In [9]:
%%sql 

SELECT (t.Milliseconds/1000) as Seconds
FROM tracks t 
WHERE (t.Milliseconds/1000) > 30
LIMIT 5;

Seconds
343.7190
342.5620
230.6190
252.0510
375.4180


### `Use `***CTE***` to improve readability`

In [14]:
%%sql 

WITH revenue_info AS (
    SELECT (t.Milliseconds / 1000) AS Seconds
    FROM tracks t 
) 
SELECT Seconds
FROM revenue_info
WHERE Seconds > 30
LIMIT 5;

Seconds
343.7190
342.5620
230.6190
252.0510
375.4180


In [19]:
%%sql 

WITH revenue_info AS (
    SELECT (t.Milliseconds / 1000) AS Seconds, 
    g.Name AS GenreName,
    SUM(i.Total) AS Revenue
    FROM tracks t
    JOIN genres g ON  t.GenreId = g.GenreId
    JOIN invoice_items ii ON t.TrackId = ii.TrackId
    JOIN invoices i ON ii.InvoiceId = i.InvoiceId
    GROUP BY Seconds, GenreName
) 
SELECT Seconds, GenreName, Revenue
FROM revenue_info
WHERE Seconds > 30
LIMIT 5;

Seconds,GenreName,Revenue
342.5620,Rock,10.89
252.0510,Rock,1.98
205.6620,Rock,3.96
210.8340,Rock,12.870000000000001
263.4970,Rock,22.82


In [22]:
%%sql 

WITH revenue_info AS (
    SELECT (t.Milliseconds / 1000) AS Seconds, 
    g.Name AS GenreName,
    SUM(i.Total) AS Revenue
    FROM tracks t
    JOIN genres g ON  t.GenreId = g.GenreId
    JOIN invoice_items ii ON t.TrackId = ii.TrackId
    JOIN invoices i ON ii.InvoiceId = i.InvoiceId
    GROUP BY Seconds, GenreName
) 
SELECT GenreName, CEIL(SUM(Revenue)) AS GenreRevenue
FROM revenue_info
WHERE Seconds > 30
GROUP BY GenreName
ORDER BY GenreRevenue DESC

GenreName,GenreRevenue
Rock,7721.0
Latin,3473.0
Metal,2094.0
Alternative & Punk,1948.0
TV Shows,818.0
Jazz,747.0
Drama,545.0
Blues,430.0
R&B/Soul,339.0
Reggae,333.0
